In [1]:
import AdventOfCode
import qualified Data.Vector as V
import qualified Data.IntMap.Strict as I

In [2]:
input <- dayString 9

instructions = read ("[" ++ input ++ "]") :: [Int]

In [3]:
data Mode = Position | Immediate | Relative deriving (Eq, Show, Enum)

data Opcode = Opcode
    { opcodeCode  :: Int
    , opcodeMode1 :: Mode
    , opcodeMode2 :: Mode
    , opcodeMode3 :: Mode
    } deriving (Eq, Show)

data ProgramState = ProgramState
    { programStateProgram      :: V.Vector Int
    , programStateMemory       :: I.IntMap Int
    , programStateRelativeBase :: Int
    } deriving (Eq, Show)

In [4]:
parseCode :: Int -> Opcode
parseCode i = let
    m3   = i `div` 10000
    m2   = (i - m3*10000) `div` 1000
    m1   = (i - m3*10000 - m2*1000) `div` 100
    code = i `rem` 100
    in Opcode code (toEnum m1) (toEnum m2) (toEnum m3)

In [5]:
access :: ProgramState -> Int -> Int
access (ProgramState program memory _) index = if index >= V.length program
    then I.findWithDefault 0 index memory
    else program V.! index

update :: ProgramState -> Int -> Int -> ProgramState
update ps@(ProgramState program memory _) index value = if index >= V.length program
    then ps{ programStateMemory = I.insert index value memory }
    else ps{ programStateProgram = program V.// [(index,value)] }

address :: Mode -> ProgramState -> Int -> Int
address mode  programState index = access programState (computeIndex mode programState index)

computeIndex :: Mode -> ProgramState -> Int -> Int
computeIndex Position  programState index = programState `access` index
computeIndex Immediate programState index = index
computeIndex Relative  programState index = (programState `access` index) + programStateRelativeBase programState

In [6]:
step :: ProgramState -> [Int] -> [Int] -> Int -> (ProgramState, [Int], [Int], Int)
step programState input output index = do
    let opcode = parseCode (programState `access` index)
        aIndex = computeIndex (opcodeMode1 opcode) programState (index+1)
        a = address (opcodeMode1 opcode) programState (index+1)
        b = address (opcodeMode2 opcode) programState (index+2)
        o = computeIndex (opcodeMode3 opcode) programState (index+3)
    case opcodeCode opcode of
        1 -> let
            p = update programState o (a+b)
            in (p, input, output, index+4)
        2 -> let
            p = update programState o (a*b)
            in (p, input, output, index+4)
        3 -> let
            (v:input') = input
            p = update programState aIndex v
            in (p, input', output, index+2)
        4 -> let
            output' = output ++ [a]
            in (programState, input, output', index+2)
        5 -> if a/=0
            then (programState, input, output, b)
            else (programState, input, output, index+3)
        6 -> if a==0
            then (programState, input, output, b)
            else (programState, input, output, index+3)
        7 -> if a < b
            then (update programState o 1, input, output, index+4)
            else (update programState o 0, input, output, index+4)
        8 -> if a == b
            then (update programState o 1, input, output, index+4)
            else (update programState o 0, input, output, index+4)
        9 -> let
            relativeBase' = programStateRelativeBase programState + a
            p = programState { programStateRelativeBase = relativeBase' }
            in (p, input, output, index+2)
        99 -> (programState, input, output, index)

In [7]:
loop program input output index = let
    (program', input', output', index') = step program input output index
    in if index == index'
        then (program', output', index')
        else loop program' input' output' index'

In [8]:
parse s = read ("[" ++ s ++ "]") :: [Int]
strings = [ "109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99"
          , "1102,34915192,34915192,7,4,7,99,0"
          , "104,1125899906842624,99"
          ]

new program = ProgramState (V.fromList program) I.empty 0

In [10]:
stepper = iterate (\(program,input,output,index) -> step program input output index)

trace = stepper (new (parse (head strings)), [], [], 0)

In [16]:
mapM_ (print . (\s -> loop (new (parse s)) [] [] 0)) strings

(ProgramState {programStateProgram = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99], programStateMemory = fromList [(100,16),(101,1)], programStateRelativeBase = 16},[109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99],15)
(ProgramState {programStateProgram = [1102,34915192,34915192,7,4,7,99,1219070632396864], programStateMemory = fromList [], programStateRelativeBase = 0},[1219070632396864],6)
(ProgramState {programStateProgram = [104,1125899906842624,99], programStateMemory = fromList [], programStateRelativeBase = 0},[1125899906842624],2)

In [17]:
loop (new (parse input)) [1] [] 0

(ProgramState {programStateProgram = [1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1101,0,3,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,904,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,1,3512778005,1101,29,0,1010,1102,1,1,1021,1101,0,36,1002,1101,573,0,1026,1101,0,33,1012,1102,1,25,1004,1102,1,38,1000,1102,31,1,1003,1102,23,1,1006,1102,777,1,1028,1102,20,1,1011,1101,0,566,1027,1101,0,27,1009,1101,26,0,1005,1101,0,0,1020,1102,1,37,1014,1101,32,0,1001,1101,0,24,1007,1101,0,35,1018,1101,30,0,1017,1101,0,22,1008,1102,460,1,1023,1101,0,768,1029,1102,1,487,1024,1102,1,34,1013,1102,1,28,1015,1101,0,39,1019,1101,478,0,1025,1101,0,463,1022,1101,21,0,1016,109,9,1208,0,30,63,1005,63,201,1001,64,1,64,1105,1,203,4,187,1002,64,2,64,109,3,1201,-8,0,63,1008,63,24,63,1005,63,227,1001,64,1,64,1106,0,229,4,209,1002,64,2,64,109,-1,2108,32,-8,63,1005,63,245,1106,0,251,4,235,1001,64,1,64,1002,64,2,64,109,-11,2101,0,2,63,1008

In [18]:
loop (new (parse input)) [2] [] 0

(ProgramState {programStateProgram = [1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1101,0,3,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,904,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,1,0,1101,29,0,1010,1102,1,1,1021,1101,0,36,1002,1101,573,0,1026,1101,0,33,1012,1102,1,25,1004,1102,1,38,1000,1102,31,1,1003,1102,23,1,1006,1102,777,1,1028,1102,20,1,1011,1101,0,566,1027,1101,0,27,1009,1101,26,0,1005,1101,0,0,1020,1102,1,37,1014,1101,32,0,1001,1101,0,24,1007,1101,0,35,1018,1101,30,0,1017,1101,0,22,1008,1102,460,1,1023,1101,0,768,1029,1102,1,487,1024,1102,1,34,1013,1102,1,28,1015,1101,0,39,1019,1101,478,0,1025,1101,0,463,1022,1101,21,0,1016,109,9,1208,0,30,63,1005,63,201,1001,64,1,64,1105,1,203,4,187,1002,64,2,64,109,3,1201,-8,0,63,1008,63,24,63,1005,63,227,1001,64,1,64,1106,0,229,4,209,1002,64,2,64,109,-1,2108,32,-8,63,1005,63,245,1106,0,251,4,235,1001,64,1,64,1002,64,2,64,109,-11,2101,0,2,63,1008,63,35,63